# Algorithm

### What algorithms do we need?

1.	Concept $\rightarrow$ Problem 

    1.	recommendation

1.	Problem $\rightarrow$ Concept  

    1.	identify

    1.	prioritization

1.	Concept $\rightarrow$ Concept 

    1.	relationship

1. Problem $\rightarrow$ Problem
    1. recommendation


## A complete procedure

* Datasets

    * **concepts_list**
        * A list of concepts, and their meanings
    * **questions_list**
        * A list of questions, with their question ID, and some other information like year and author
    * **concepts_graph**
        * a dictionary of forward links and back links of each concept.
    * **questions_concepts_matrix**
        * If a concept is mensioned in a question, assign 1. If not, assign 0.
    


* Query a question

    1. User import a image or pdf
        1. Use OCR to convert it into text
        1. Process data 
    
    1. Match the question with our **concepts_list**
        1. If a concept is included, assign 1. If not, assign 0.
        1. For all the concepts included, find a path in **concepts_graph**
        1. Print out the path and concepts
        
    1. Recommend similar questions.
        1. For all the questions in the **questions_list**, calculate the distance between them and the target question.
        1. Print out the ones have minimal distance.



* Query concepts

    1. User import several concepts
        1. Give dynamic suggestions
        
    1. Print out the questions including these concepts
        1. Weighted score
    


* Add a question
    
    1. User import a image or pdf
        1. Use OCR to vonvert it into text(Latex?)
    
    1. Detect the concepts in the question
    
    1. Assign rights to user to edit the text and concepts
    
    
* Add a concept

    1. User import a concept
        1. Give similar concepts suggestions
        
    1. Assign rights to user to edit the concept and its meaning
    
    1. Assign rights to user to choose the backlink and forwardlink of that concept




        

## Query a question
### 1. User: Input a image or pdf

* OCR
* NLP: stopwords, punctuation... (Math?)

In [1]:
from PIL import Image
import pytesseract as pt
import numpy as np
import pandas as pd

import nltk
from nltk.tokenize import word_tokenize
from collections import Counter
#nltk.download('wordnet')      #download if using this module for the first time

from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
#nltk.download('stopwords')    #download if using this module for the first time

#For Gensim
import gensim
import string
from gensim import corpora
from gensim.corpora.dictionary import Dictionary
from nltk.tokenize import word_tokenize

In [2]:
# OCR converts image to text

img1 = Image.open("page2.png")
ques1 = pt.image_to_string(img1)
print(ques1)

Problem 3

Recall that the th tted value ¥; can be expressed as a linear combination ofthe response
values, ie,

   

Lh

 

Problem 4

Consider the regression through the origin model given by

a Yiasut+a@ i=1,2...n  @#N(0,0%)
‘The estimated model at observed point (2, y) is

g= Be,

where
2)

‘Complete the following tasks
i. Show that

 

is an unbiased estimator of
ii, Compute the standard error of estimator 2.
iii, Identify the probability distribution of estimator 3.

2


In [3]:
# Process data

stopwords = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()

def clean(document):
    stopwordremoval = " ".join([i for i in document.lower().split() if i not in stopwords])
    punctuationremoval = ''.join(ch for ch in stopwordremoval if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punctuationremoval.split())
    return normalized


In [4]:
sample1 = "Our board of directors boasts 11 seasoned technology and business leaders from Adobe, GSK, HGGC and more."
sample2 = "Our executives lead by example and guide us to accomplish great things every day."
sample3 = "Working at Pluralisght means being surrounded by smart, passionate people who inspire us to do our best work."
sample4 = "A leadership team with vision."
sample5 = "Courses on cloud, microservices, machine learning, security, Agile and more."
sample6 = "Interactive courses and projects."
sample7 = "Personalized course recommendations from Iris."
sample8 = "We’re excited to announce that Pluralsight has ranked #9 on the Great Place to Work 2018, Best Medium Workplaces list!"
sample9 = "Few of the job opportunities include Implementation Consultant - Analytics, Manager - assessment production, Chief Information Officer, Director of Communications."

# compile documents
compileddoc = [sample1, sample2, sample3, sample4, sample5, sample6, sample7, sample8, sample9] 

final_doc = [clean(document).split() for document in compileddoc]
print("Before text-cleaning:", compileddoc[0]) 

Before text-cleaning: Our board of directors boasts 11 seasoned technology and business leaders from Adobe, GSK, HGGC and more.


In [5]:
print("After text-cleaning:",final_doc[0])

After text-cleaning: ['board', 'director', 'boast', '11', 'seasoned', 'technology', 'business', 'leader', 'adobe', 'gsk', 'hggc', 'more']


### 2. Match the question with our concepts_list

* Use LDA to detect the topics in the question (*Really need it?*)
    
* Compare the LDA topics with concepts_list
    
* If the topic is included in concepts_list, add a tag. If not, create a new concepts    

In [6]:
# LDA
dictionary = corpora.Dictionary(final_doc)

DT_matrix = [dictionary.doc2bow(doc) for doc in final_doc]

Lda_object = gensim.models.ldamodel.LdaModel

lda_model_1 = Lda_object(DT_matrix, num_topics=2, id2word = dictionary)

print(lda_model_1.print_topics(num_topics=2, num_words=5))


[(0, '0.029*"course" + 0.028*"u" + 0.025*"great" + 0.021*"work" + 0.019*"lead"'), (1, '0.026*"director" + 0.024*"course" + 0.021*"best" + 0.021*"more" + 0.020*"gsk"')]


In [7]:
lda_model_2 = Lda_object(DT_matrix, num_topics=3, id2word = dictionary)

print(lda_model_2.print_topics(num_topics=3, num_words=5))

[(0, '0.034*"course" + 0.034*"more" + 0.032*"director" + 0.020*"learning" + 0.020*"security"'), (1, '0.031*"best" + 0.031*"work" + 0.030*"u" + 0.030*"surrounded" + 0.029*"smart"'), (2, '0.035*"great" + 0.033*"ranked" + 0.033*"work" + 0.033*"we’re" + 0.033*"excited"')]


Find the shortest path in **concepts_graph**

In [8]:
# Random questions_concepts_matrix

concepts_list = ["con1", "con2", "con3", "con4", "con5", "con6", "con7", "con8", "con9", "con10"]
ques_list = ["quesID1", "quesID2", "quesID3", "quesID4", "quesID5", "quesID6", "quesID7", "quesID8", "quesID9", "quesID10"]

np.random.seed(0)
ques_concepts = pd.DataFrame(data = np.random.binomial(size = 100, n = 1, p = 0.2).reshape(10, 10), 
                            index = ques_list, 
                            columns = concepts_list)
print(ques_concepts)


          con1  con2  con3  con4  con5  con6  con7  con8  con9  con10
quesID1      0     0     0     0     0     0     0     1     1      0
quesID2      0     0     0     1     0     0     0     1     0      1
quesID3      1     0     0     0     0     0     0     1     0      0
quesID4      0     0     0     0     0     0     0     0     1      0
quesID5      0     0     0     0     0     0     0     0     0      0
quesID6      0     0     1     0     0     0     0     0     0      0
quesID7      0     0     0     0     0     0     1     0     1      0
quesID8      1     0     1     0     0     0     0     0     0      0
quesID9      0     0     0     0     0     0     0     0     0      1
quesID10     0     0     0     0     0     0     0     0     1      0


In [9]:
# A random concepts_graph

concepts_graph = {"con1": ["con3", "con5", "con7", "con9"],
                  "con2": ["con4"],
                  "con3": ["con5"],
                  "con4": ["con6"],
                  "con5": ["con8"],
                  "con6": [],
                  "con7": ["con10"],
                  "con8": ["con10"],
                  "con9": [],
                  "con10": []}

print(concepts_graph)

{'con1': ['con3', 'con5', 'con7', 'con9'], 'con2': ['con4'], 'con3': ['con5'], 'con4': ['con6'], 'con5': ['con8'], 'con6': [], 'con7': ['con10'], 'con8': ['con10'], 'con9': [], 'con10': []}


In [10]:
# Find shortest path functions

def findShortestPath(graph, start, end, path=[]):
    path = path + [start]
    if start == end:
        return path
    
    shortestPath = []
    for node in graph[start]:
        if node not in path:
            newpath = findShortestPath(graph, node, end, path)
            if newpath:
                if not shortestPath or len(newpath)<len(shortestPath):
                    shortestPath = newpath
    return shortestPath


def findConPath (quesID, ques_concepts, concepts_mat):
    concepts = np.array(ques_concepts.loc[quesID, ques_concepts.loc[quesID] == 1].index)
    n_con = len(concepts)
    
    path_list = []
    for start in concepts:
        for end in concepts[concepts != start]:
            path = findShortestPath(concepts_graph, start, end, path_list)
            if path:
                path_list.append(path)
    return(path_list)


In [11]:
# The concepts path of quesID3
quesID3_path = findConPath("quesID3", ques_concepts, concepts_graph)
print(quesID3_path)   

[['con1', 'con5', 'con8']]


### 3. Recommend similar questions.

* How to define distance?

**Distance definition**

Minkowski distance

$$
dist(X, Y) = \left( \sum_{i=1}^n |x_i - y_i|^p \right)^{\frac{1}{p}}
$$


Jaccard similarity coefficient $\surd$

$$
J(A,B) = \frac{|A \cap B|}{|A \cup B|}
$$


Pearson correlation coefficient

$$
\rho_{XY} = \frac{\sum_{i=1}^n (X_i - \mu_X)(Y_i - \mu_Y)}{\sqrt{\sum_{i=1}^n (X_i - \mu_X)^2}\sqrt{\sum_{i=1}^n (Y_i - \mu_Y)^2}}
$$

In [12]:
# Distance between questions
def distBtwQues(quesID1, quesID2, ques_concepts):
    ques1_con = ques_concepts.loc[quesID1]
    ques2_con = ques_concepts.loc[quesID2]
    if sum((ques1_con + ques2_con) != 0) != 0:
        Jdist = 1 - sum(ques1_con * ques2_con != 0) /sum((ques1_con + ques2_con) != 0)
    else:
        Jdist = 1
    return(Jdist)


# Find similar questions
def findSimQues(quesID, ques_concepts):
    ques_dis = []
    for quesID2 in ques_concepts.index[ques_concepts.index != quesID]:
        ques_dis.append(distBtwQues(quesID, quesID2, ques_concepts))
    sort_index = np.argsort(ques_dis)
    return(ques_concepts.index[sort_index])

In [13]:
# Questions distances

ques_dis_mat = pd.DataFrame(index = ques_list, columns = ques_list)
for quesID1 in ques_list:
    for quesID2 in ques_list:       
        ques_dis_mat.loc[quesID1, quesID2] = distBtwQues(quesID1, quesID2, ques_concepts)
print(ques_dis_mat)


           quesID1   quesID2   quesID3 quesID4 quesID5 quesID6   quesID7  \
quesID1          0      0.75  0.666667     0.5       1       1  0.666667   
quesID2       0.75         0      0.75       1       1       1         1   
quesID3   0.666667      0.75         0       1       1       1         1   
quesID4        0.5         1         1       0       1       1       0.5   
quesID5          1         1         1       1       1       1         1   
quesID6          1         1         1       1       1       0         1   
quesID7   0.666667         1         1     0.5       1       1         0   
quesID8          1         1  0.666667       1       1     0.5         1   
quesID9          1  0.666667         1       1       1       1         1   
quesID10       0.5         1         1       0       1       1       0.5   

           quesID8   quesID9 quesID10  
quesID1          1         1      0.5  
quesID2          1  0.666667        1  
quesID3   0.666667         1        1  
que

In [14]:
# Similar questions for quesID1
quesID1_min = findSimQues("quesID1", ques_concepts)
print(quesID1_min)


Index(['quesID3', 'quesID9', 'quesID2', 'quesID6', 'quesID1', 'quesID4',
       'quesID5', 'quesID7', 'quesID8'],
      dtype='object')


## Query concepts

### 1. User import several concepts

* Approximate string matching    

### 2. Print out the questions including these concepts

* Distance between question and concepts list
    
* Need to be weighted


In [15]:
# Distance of all questions to the input concepts list
def distToCons(input_cons, ques_concepts):
    cons_list = np.zeros(len(ques_concepts.columns))
    ques0 = pd.Series(cons_list, index = ques_concepts.columns)
    ques0[input_cons] = 1
    
    ques_dist = []
    for ques in np.array(ques_concepts.index):
        ques_con = ques_concepts.loc[ques]
        if sum((ques_con + ques0) != 0) != 0:
            Jdist = 1 - sum(ques_con * ques0 != 0) /sum((ques_con + ques0) != 0)
        else:
            Jdist = 1
        ques_dist.append(Jdist)
        
    sort_index = np.argsort(ques_dist)
    return(ques_concepts.index[sort_index])


In [16]:
# Suppose that input "con1" and "con8"
input_cons = ["con1", "con8"]

recom_ques = distToCons(input_cons, ques_concepts)
print(recom_ques)

Index(['quesID3', 'quesID1', 'quesID8', 'quesID2', 'quesID4', 'quesID5',
       'quesID6', 'quesID7', 'quesID9', 'quesID10'],
      dtype='object')
